In [30]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [28]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from joblib import dump
from joblib import load

In [16]:
def make_dataset(filename="names_train.csv"):
    return pd.read_csv(filename)

In [17]:
make_dataset(filename="names_train.csv")

,video_name,is_comic
0,Le Barbecue Disney - La chanson de Frédéric Fr...,1
1,Le Roi et l'Oiseau - La Chronique de Christine...,1
2,L'amour du lac - La chronique d'Hippolyte Gira...,1
3,La fille de la piscine de Léa Tourret - La chr...,0
4,"""Le soleil va moins faire son malin quand Jean...",0
...,...,...
994,Au Ces de Las Vegas des innovations à gogo et ...,0
995,Le questionnaire Jupiproust de Frédérique Matonti,1
996,Raphael - La Nuit N'en Finit Plus (cover),0
997,Des sachets de drogue estampillés Haribeu sais...,1


In [27]:
def create_pipeline(model_type='linear', output_model_path='trained_model.joblib'):
    # Étape 1 : Charger le dataset
    df = make_dataset("names_train.csv")

    # Étape 2 : Séparer les données en ensembles d'entraînement et de test
    X = df["video_name"]  # Supposons que la colonne contenant les titres s'appelle 'title'
    y = df["is_comic"]  # Supposons que la cible s'appelle 'is_comic'

    # Diviser les données en 80% pour l'entraînement et 20% pour le test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Étape 3 : Choisir le modèle en fonction du type spécifié
    if model_type == 'linear':
        classifier = LogisticRegression()
    elif model_type == 'random_forest':
        classifier = RandomForestClassifier()
    else:
        raise ValueError("Le type de modèle doit être 'linear' ou 'random_forest'.")

    # Étape 4 : Créer la pipeline de transformation et d'entraînement
    pipeline = Pipeline([
        ('vectorizer', CountVectorizer()),  # Transformer les titres en mots encodés (one-hot encoding)
        ('classifier', classifier)  # Modèle choisi (régression logistique ou forêt aléatoire)
    ])

    # Étape 5 : Entraîner le modèle sur les données d'entraînement
    pipeline.fit(X_train, y_train)
    print(f"Entraînement du modèle {model_type} terminé.")

    # Étape 6 : Sauvegarder le modèle entraîné dans un fichier .joblib
    dump(pipeline, output_model_path)
    print(f"Modèle sauvegardé dans le fichier : {output_model_path}")

    # Retourner la pipeline pour une éventuelle utilisation ultérieure
    return pipeline

# Exemple d'utilisation : créer et entraîner une pipeline avec un modèle linéaire
create_pipeline(model_type='linear', output_model_path='linear_model.joblib')

# Exemple d'utilisation : créer et entraîner une pipeline avec un modèle Random Forest
create_pipeline(model_type='random_forest', output_model_path='random_forest_model.joblib')

Entraînement du modèle linear terminé.
Modèle sauvegardé dans le fichier : linear_model.joblib
Entraînement du modèle random_forest terminé.
Modèle sauvegardé dans le fichier : random_forest_model.joblib


Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('classifier', RandomForestClassifier())])

PIPELINE PREDICT

In [29]:
# Fonction pour charger le modèle sauvegardé et faire des prédictions sur de nouveaux titres
def predict_with_model(model_path='trained_model.joblib', new_titles=['Exemple de titre de vidéo']):
    # Étape 1 : Charger le modèle entraîné à partir du fichier .joblib
    try:
        pipeline = load(model_path)
        print(f"Modèle chargé depuis : {model_path}")
    except FileNotFoundError:
        print(f"Fichier de modèle '{model_path}' non trouvé. Veuillez vérifier le chemin.")
        return None

    # Étape 2 : Faire des prédictions sur les nouveaux titres de vidéos
    predictions = pipeline.predict(new_titles)

    # Afficher les titres avec leurs prédictions
    for title, prediction in zip(new_titles, predictions):
        print(f"Titre: '{title}' --> Prédiction: {'comique' if prediction else 'non comique'}")

    return predictions

# Exemple d'utilisation de la fonction de prédiction
predict_with_model(
    model_path='linear_model.joblib', 
    new_titles=['Ma chronique préférée', 'Vidéo de stand-up hilarante', 'Analyse sérieuse d’un sujet politique']
)


Modèle chargé depuis : linear_model.joblib
Titre: 'Ma chronique préférée' --> Prédiction: non comique
Titre: 'Vidéo de stand-up hilarante' --> Prédiction: non comique
Titre: 'Analyse sérieuse d’un sujet politique' --> Prédiction: non comique


array([0, 0, 0], dtype=int64)